

---

## 1. 整体思路梳理

1. **原有流程**
    - 由 `calOneLayerDoseOld` 生成稀疏矩阵 (CSC) 形式的 spot→voxel 贡献。
    - `calDoseSumSquares` 在 GPU 上把每个 spot 的贡献按 ROI 累加（只累加到 ROI 列表里）。
    - 最后做一次 `sqrt`。
2. **scatter + superposition 流程**
    - **Scatter 阶段**：对每个 spot，直接在它影响范围内的所有体素（通常是一个 3D tile）上按距离算出权重并写入全局剂量缓冲。
    - **Superposition 阶段**：把所有 spot 的“局部散射贡献”叠加起来（直接加到全局剂量图上即可），不再需要 CSC 稀疏码。
3. **Tile Batch**
    - 把 spot 按批次（batch）分组，每批 `T` 个 spot，launch 一个 scatter kernel。
    - 每个 block 负责一个 spot 的一个 tile（tile 大小可调，比如 16×16×16 或 32×32×8），tile 内的线程并行给体素写加权剂量。
    - kernel 里用 shared memory 载入 spot 的位置／能量参数，以及预先加载好的核函数表（如果有的话）。

---

## 2. 核心 CUDA Kernel 示例

```cpp
// scatter_kernel.cu

// 每个 thread 处理 tile 内一个 voxel
// inputs:
//   doseGrid    : Grid(3D) 信息
//   d_doseVol   : 全局剂量数组，大小 gridSize = X*Y*Z
//   spotPosXYZ  : 每个 spot 的位置 (x,y,z) in index space
//   spotEnergy  : spot 的能量，用于核函数查表
//   kernelTable : 预计算好的距离-权重一维表，长度 KERNEL_SIZE
//   kernelRadiusInVoxels : 核函数最大半径 (voxels)
// parameters:
//   nSpotsThisBatch: 本次 batch 中 spot 数量
//   tileDimX/Y/Z   : tile 尺寸
extern "C"
__global__
void scatter_and_superpose(
    float* __restrict__ d_doseVol,
    int gridDimX, int gridDimY, int gridDimZ,
    const float3* __restrict__ spotPosXYZ,
    const float*  __restrict__ spotEnergy,
    const float*  __restrict__ kernelTable,
    int kernelRadius,
    int nSpotsThisBatch
) {
    // blockIdx.x: [0, nSpotsThisBatch)
    int spotIdx = blockIdx.x;
    if (spotIdx >= nSpotsThisBatch) return;

    // 每个 block 都为一个 spot 处理一个 tile
    int tileOriginX = (threadIdx.z) * tileDimX;
    int tileOriginY = (threadIdx.y) * tileDimY;
    int tileOriginZ = (threadIdx.x) * tileDimZ;
    // 计算全局 voxel index
    int vx = tileOriginX + threadIdx.x;
    int vy = tileOriginY + threadIdx.y;
    int vz = tileOriginZ + threadIdx.z;
    if (vx >= gridDimX || vy >= gridDimY || vz >= gridDimZ) return;

    float3 spotPos = spotPosXYZ[spotIdx];
    // 计算体素中心到 spot 的距离（以 voxel 单位）
    float dx = (vx + 0.5f) - spotPos.x;
    float dy = (vy + 0.5f) - spotPos.y;
    float dz = (vz + 0.5f) - spotPos.z;
    float dist = sqrtf(dx*dx + dy*dy + dz*dz);
    if (dist > kernelRadius) return;

    // 查表获得权重
    int idx = min(int(dist), kernelRadius);
    float weight = kernelTable[idx] * spotEnergy[spotIdx];

    // 原子加到全局剂量
    // d_doseVol 的索引： (vz*gridDimY + vy)*gridDimX + vx
    int gIdx = (vz * gridDimY + vy) * gridDimX + vx;
    atomicAdd(&d_doseVol[gIdx], weight);
}

```

> 说明：
>
> - `kernelTable` 在 host 端预先用 RayTraceDicom 里那种 Gaussian / scatter 模型算好，长度 = `maxRadius + 1`。
> - `spotEnergy` 可以直接拿你原来 `layerEnergy` 里的值。
> - `spotPosXYZ` 由原来 `idbeamxy` 数组（V 平面上子束位置）换算为 voxel-space 坐标。

---

## 3. 在现有代码中插入 Scatter + Superposition

以你原来 `cudaCalDoseNorm` 中的循环为例，示意如何替换 `calDoseSumSquares` 调用：

```diff
--- a/cudaCalDose.cpp
+++ b/cudaCalDose.cpp
@@ EXPORT
 int cudaCalDoseNorm(...)
 {
     // ... 前面初始化、texture 创建、省略 ...
-    for (int i = 0; i < layerInfo.size(); i++) {
+    // 先把全局剂量置零
+    checkCudaErrors(cudaMemset(d_doseNorm, 0, sizeof(float) * nRoi));
+
+    // 从 layerInfo 构造每个 spot 在 voxel-space 的 float3 数组
+    std::vector<float3>  h_spotPos; h_spotPos.reserve(totalSpots);
+    std::vector<float>   h_spotEne; h_spotEne.reserve(totalSpots);
+    // … 用原来的 layerInfo & idbeamxy 把 spotPosXYZ / spotEnergy 填满 …
+
+    // 拷到 GPU
+    float3* d_spotPosXYZ;
+    float*  d_spotEnergy;
+    checkCudaErrors(cudaMalloc(&d_spotPosXYZ, totalSpots * sizeof(float3)));
+    checkCudaErrors(cudaMalloc(&d_spotEnergy,  totalSpots * sizeof(float)));
+    checkCudaErrors(cudaMemcpy(d_spotPosXYZ, h_spotPos.data(), totalSpots*sizeof(float3), cudaMemcpyHostToDevice));
+    checkCudaErrors(cudaMemcpy(d_spotEnergy, h_spotEne.data(), totalSpots*sizeof(float),  cudaMemcpyHostToDevice));
+
+    // 预先拷入 kernelTable
+    float* d_kernelTable;
+    checkCudaErrors(cudaMalloc(&d_kernelTable, (kernelRadius+1)*sizeof(float)));
+    checkCudaErrors(cudaMemcpy(d_kernelTable, h_kernelTable.data(), (kernelRadius+1)*sizeof(float), cudaMemcpyHostToDevice));
+
+    // 按批次 launch scatter kernel
+    int spotsLeft = totalSpots, offset = 0;
+    while (spotsLeft > 0) {
+        int batch = min(spotsLeft, SPOTS_PER_BATCH);
+        dim3 grid(batch), block(tileDimZ, tileDimY, tileDimX);
+        scatter_and_superpose<<<grid, block>>>(
+            d_doseNorm, gridDimX, gridDimY, gridDimZ,
+            d_spotPosXYZ + offset,
+            d_spotEnergy  + offset,
+            d_kernelTable, kernelRadius,
+            batch
+        );
+        spotsLeft -= batch;
+        offset   += batch;
+    }
+
+    // 最后做 ROI 索引提取，把 d_doseNorm （大体积）里 ROI 点拷回 h_doseNorm.ptr
+    // 这一步替代原来 calDoseSumSquares + calSqrt
+    copyROIsFromVolume(d_doseNorm, h_roiIndex, (float*)h_doseNorm.ptr);
+
     // … 原来的 free / destroy texture 等 …
 }

```

- **`SPOTS_PER_BATCH`**、**`tileDimX/Y/Z`**、**`kernelRadius`** 等宏/常量需根据显存和性能调优。
- `copyROIsFromVolume` 可以写成一个简单的 CUDA kernel，把全体积 `d_doseNorm` 中 ROI 索引对应值摘出到输出数组。

---

## 4. superposition 细节

- 我在 kernel 里直接用 `atomicAdd` 做叠加，是最简单的 superposition。
- 如果你希望更高效，可以把每个 block 的 tile 先累加到 shared memory 再一次性写回全局（减少原子冲突）；但思路相同：每个 spot 贡献一个局部 3D tile，最后都累到同一个全局剂量图里。

---

### 小结

1. **移除 CSC + 稀疏矩阵**，用 Scatter kernel 直接给每个 voxel 写值。
2. **批次（batch）＋ tile**：每个 block 负责一个 spot 的一个 tile，thread 负责 tile 内一个 voxel。
3. **Kernel Table**：预先在 host 端生成 Gaussian/scatter 权重查表，kernel 里查表再乘能量。
4. **接口不变**：`cudaCalDoseNorm`、`cudaFinalDose` …外部 Python 层调用接口保持不变，所有传入参数都沿用；我们只在内部替换核心计算流程。

这样，你就把原来“先叠加 spot → 稀疏矩阵 → 再累加 ROI”改成了“scatter → superposition → 再抽 ROI”。后续可进一步优化共享存储、层间并发等。希望这个示例能帮你在现有框架里快速落地！

下面是把之前省略的部分都补全后的示例代码，完整演示了：

1. **从原来的 `layerInfo`、`layerEnergy`、`idbeamxy`、`bmxDir`/`bmyDir`、`sourcePos` 等输入数组** 构造每个 spot 在体素坐标系下的浮点位置 `h_spotPos` 和对应的能量 `h_spotEne`。
2. **在 host 端生成 Gaussian scatter 核函数查表** `h_kernelTable`（这里用一个简单的高斯模型作为例子）。
3. **在设备端分批（tile‐batch）launch `scatter_and_superpose` kernel**，把每个 spot 的局部散射贡献原子加到全局剂量体积 `d_doseVol`。
4. **用 `copyROIsFromVolume` kernel** 把全局体积上 ROI 点提取回输出 `h_doseNorm`，替代原来的 CSC + `calSqrt` 流程。

> 注意：以下示例假定你已经在函数开头完成了对 ctGrid、sourcePos、bmxDir、bmyDir、h_layerInfo、h_layerEnergy、h_idbeamxy、h_roiIndex 等 pybind11 数组的 <…>.request() 并拷贝到对应的 host 变量里。
>

```cpp
// ---------------------------------------------
// scatter kernel: 每个 block 处理一个 spot 的一个 tile
// tileDimX, tileDimY, tileDimZ 可调
constexpr int tileDimX = 16;
constexpr int tileDimY = 16;
constexpr int tileDimZ = 4;

extern "C" __global__
void scatter_and_superpose(
    float* __restrict__ d_doseVol,
    int gridDimX, int gridDimY, int gridDimZ,
    const float3* __restrict__ spotPosXYZ,
    const float*  __restrict__ spotEnergy,
    const float*  __restrict__ kernelTable,
    int   kernelRadius,
    int   nSpotsThisBatch
) {
    int spotIdx = blockIdx.x;
    if (spotIdx >= nSpotsThisBatch) return;

    // 线程三维索引对应 tile 内 voxel 坐标
    int lx = threadIdx.x;
    int ly = threadIdx.y;
    int lz = threadIdx.z;

    int vx = lx + (blockIdx.y * tileDimX);
    int vy = ly + (blockIdx.z * tileDimY);
    int vz = lz;  // 假设我们沿 Z 分层

    if (vx >= gridDimX || vy >= gridDimY || vz >= gridDimZ) return;

    float3 spot = spotPosXYZ[spotIdx];
    // 体素中心到 spot 的距离 (vox 单位)
    float dx = (vx + 0.5f) - spot.x;
    float dy = (vy + 0.5f) - spot.y;
    float dz = (vz + 0.5f) - spot.z;
    float dist = sqrtf(dx*dx + dy*dy + dz*dz);
    if (dist > kernelRadius) return;

    int idx = min(int(dist), kernelRadius);
    float w = kernelTable[idx] * spotEnergy[spotIdx];

    int gIdx = (vz * gridDimY + vy) * gridDimX + vx;
    atomicAdd(&d_doseVol[gIdx], w);
}

// 提取 ROI 的 kernel
extern "C" __global__
void copyROIsKernel(
    const float* __restrict__ d_volume,
    const int3*  __restrict__ d_roiIdx,
    float*       __restrict__ d_outDose,
    int          nRoi,
    int gridDimX, int gridDimY
) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= nRoi) return;
    int3 v = d_roiIdx[i];
    int idx = (v.z * gridDimY + v.y) * gridDimX + v.x;
    d_outDose[i] = d_volume[idx];
}

// --------------- cudaCalDoseNorm 改写示例 ---------------
EXPORT
int cudaCalDoseNorm(
    pybind11::array doseNorm,     // out: 大小 nRoi
    /* … 省略其它传参 … */
) {
    // --- 前置：把所有 pybind11 数组 cast & request 到 h_XXX，并得到：
    //    int    nRoi;
    //    int3   *h_roiIndex.ptr;
    //    vec3f  sourcePos, bmxDir, bmyDir;
    //    Grid   ctGrid;            // 包含 corner, resolution, dims{x,y,z}
    //    int    nLayers;
    //    int   *h_layerInfo.ptr;    // 每层 spot 数量
    //    float *h_layerEnergy.ptr;  // 每层物理能量
    //    float *h_idbeamxy.ptr;     // 2 floats * totalSpots (x,y)
    //    float *h_doseNorm.ptr;     // 输出 buffer
    //
    // 这里略去 cast/request 代码，假定 host 变量已就绪。

    int gridDimX = ctGrid.dims.x,
        gridDimY = ctGrid.dims.y,
        gridDimZ = ctGrid.dims.z;

    // 1) 计算 totalSpots
    int totalSpots = 0;
    for (int i = 0; i < nLayers; ++i) {
        totalSpots += h_layerInfo.ptr[i];
    }

    // 2) 构造 host 端 spotPosXYZ & spotEnergy
    std::vector<float3> h_spotPos;  h_spotPos.reserve(totalSpots);
    std::vector<float>  h_spotEne;  h_spotEne.reserve(totalSpots);

    int offset = 0;
    for (int lay = 0; lay < nLayers; ++lay) {
        int layerSize = h_layerInfo.ptr[lay];
        float enePhys  = h_layerEnergy.ptr[lay];  // 直接用物理能量作为权重
        for (int j = 0; j < layerSize; ++j) {
            // 从 idbeamxy 拿出这个 spot 在 V 平面上的 x/y 偏移
            float dx = h_idbeamxy.ptr[2*(offset+j) + 0];
            float dy = h_idbeamxy.ptr[2*(offset+j) + 1];
            // 世界坐标 = sourcePos + bmxDir*dx + bmyDir*dy
            float3 wpos;
            wpos.x = sourcePos.x + bmxDir.x * dx + bmyDir.x * dy;
            wpos.y = sourcePos.y + bmxDir.y * dx + bmyDir.y * dy;
            wpos.z = sourcePos.z + bmxDir.z * dx + bmyDir.z * dy;
            // 转到体素索引空间
            float3 vpos;
            vpos.x = (wpos.x - ctGrid.corner.x) / ctGrid.resolution.x;
            vpos.y = (wpos.y - ctGrid.corner.y) / ctGrid.resolution.y;
            vpos.z = (wpos.z - ctGrid.corner.z) / ctGrid.resolution.z;
            h_spotPos.push_back(vpos);
            h_spotEne.push_back(enePhys);
        }
        offset += layerSize;
    }

    // 3) 构造 Gaussian 核函数查表 (radius = 3σ，σ 这里取 2 voxels 为例)
    const int kernelRadius = int(3 * 2.0f);
    std::vector<float> h_kernelTable(kernelRadius+1);
    float sigma = 2.0f;
    for (int d = 0; d <= kernelRadius; ++d) {
        h_kernelTable[d] = expf(-0.5f * (d*d)/(sigma*sigma));
    }

    // 4) 在 GPU 上开辟缓冲
    float3* d_spotPosXYZ = nullptr;
    float*  d_spotEnergy = nullptr;
    float*  d_kernelTable= nullptr;
    float*  d_doseVol    = nullptr;
    int3*   d_roiIdx     = nullptr;

    cudaMalloc(&d_spotPosXYZ, totalSpots * sizeof(float3));
    cudaMalloc(&d_spotEnergy, totalSpots * sizeof(float));
    cudaMalloc(&d_kernelTable,kernelRadius+1 * sizeof(float));
    cudaMalloc(&d_doseVol,    gridDimX*gridDimY*gridDimZ * sizeof(float));
    cudaMalloc(&d_roiIdx,     nRoi * sizeof(int3));

    cudaMemcpy(d_spotPosXYZ,  h_spotPos.data(),    totalSpots*sizeof(float3), cudaMemcpyHostToDevice);
    cudaMemcpy(d_spotEnergy,  h_spotEne.data(),    totalSpots*sizeof(float),  cudaMemcpyHostToDevice);
    cudaMemcpy(d_kernelTable, h_kernelTable.data(),(kernelRadius+1)*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_roiIdx,      h_roiIndex.ptr,      nRoi*sizeof(int3),              cudaMemcpyHostToDevice);

    // 5) 清零全局剂量
    cudaMemset(d_doseVol, 0, gridDimX*gridDimY*gridDimZ * sizeof(float));

    // 6) 分批 launch scatter kernel
    const int SPOTS_PER_BATCH = 256;
    int spotsLeft = totalSpots, base = 0;
    while (spotsLeft > 0) {
        int batch = std::min(spotsLeft, SPOTS_PER_BATCH);
        // grid.x = batch，grid.y/z 用来枚举 tile 在 XY 面上的分块
        dim3 gridDim(batch,
                     (gridDimX + tileDimX-1)/tileDimX,
                     (gridDimY + tileDimY-1)/tileDimY);
        dim3 blockDim(tileDimX, tileDimY, tileDimZ);

        scatter_and_superpose<<<gridDim, blockDim>>>(
            d_doseVol,
            gridDimX, gridDimY, gridDimZ,
            d_spotPosXYZ + base,
            d_spotEnergy + base,
            d_kernelTable, kernelRadius,
            batch
        );
        base      += batch;
        spotsLeft -= batch;
    }

    // 7) 用 kernel 把 ROI 点拷回 host
    float* d_outDose = nullptr;
    cudaMalloc(&d_outDose, nRoi * sizeof(float));
    {
        int thr = 256;
        int blk = (nRoi + thr - 1)/thr;
        copyROIsKernel<<<blk, thr>>>(
            d_doseVol,
            d_roiIdx,
            d_outDose,
            nRoi,
            gridDimX, gridDimY
        );
    }
    cudaMemcpy(h_doseNorm.ptr, d_outDose, nRoi*sizeof(float), cudaMemcpyDeviceToHost);

    // 8) 清理
    cudaFree(d_spotPosXYZ);
    cudaFree(d_spotEnergy);
    cudaFree(d_kernelTable);
    cudaFree(d_doseVol);
    cudaFree(d_roiIdx);
    cudaFree(d_outDose);

    return 1;
}

```

**要点回顾：**

- **位置计算**：直接把 spot 做世界坐标→体素坐标的映射，替代原来 CSC 稀疏矩阵那套流程。
- **Scatter Kernel**：每个 block 负责一个 spot 的一个 tile，线程并行给 tile 内的 voxel 原子加权。
- **Superposition**：用 `atomicAdd` 汇聚到全局剂量体积。
- **ROI 提取**：最后一个小 kernel 一次性把感兴趣的 ROI 点提取回输出。

这样就完成了“scatter + superposition + tile batch”思路的完整示例，且**不改动任何 Python/pybind11 层接口**，所有输入数组都保持不变。希望对你在现有项目中落地有帮助！